In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
num_input = 3
num_output = 3

num_hidden1 = 16
num_hidden2 = 16

my_rmse_list = []

In [3]:

session = tf.Session()

x = tf.placeholder(tf.float32, shape=[None, num_input])

y_true = tf.placeholder(tf.float32, shape=[None, num_output], name='y_true')
y_pred = tf.placeholder(tf.float32, shape=[None, num_output])


In [4]:
data_csv = pd.read_csv('../myCSVData.csv')
X = data_csv[['in1','in2','in3']]
y = data_csv[['out1','out2','out3']]
#X, y
print (type(X), type(y))
#y

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [5]:

X_a = X.values
y_a = y.values
y_dumb = X_a*2

In [6]:

def cost_func(layer, my_y_true):
    ret = tf.sqrt(tf.reduce_mean(tf.square(layer - my_y_true)))
    
    return ret

In [7]:
def nn_model_fn(features, labels, mode):
    global my_rmse_list
    # Neural Network Structure
    # Input Layer
    input_layer = tf.reshape(features["x"],[-1, 3]) 
    
    # Hidden Layer #1
    h1 = tf.layers.dense(inputs=input_layer, 
                         units=num_hidden1,
                         use_bias=True,
                         activation=tf.nn.relu)

    # Hidden Layer #2
    h2 = tf.layers.dense(inputs=h1, 
                         units=num_hidden2,
                         use_bias=True,
                         activation=tf.nn.relu)

    # Output Layer
    output_layer = tf.layers.dense(inputs=h2, 
                         units=num_output,
                         use_bias=True,
                         activation=None)
    predictions = {
        "output" : output_layer
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    cost = cost_func(output_layer, labels)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)
        
        train_op = optimizer.minimize(
                    loss=cost,
                    global_step=tf.train.get_global_step())
        #my_rmse_list += [cost.eval]
        return tf.estimator.EstimatorSpec(mode=mode, loss=cost, train_op=train_op)
    
    eval_metric_ops = {
        "error":tf.metrics.mean_squared_error(
        labels=labels, predictions=predictions["output"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=cost, eval_metric_ops=eval_metric_ops)

In [8]:
my_nn_esti = tf.estimator.Estimator(model_fn=nn_model_fn, model_dir="./sav")

tensors_to_log = {"RMSE": "output"}
rmse_list = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './sav', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc787cd4198>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [9]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                    #x={"in1" : X_a[:,0], "in2":X_a[:,1], "in3":X_a[:,2]},
                    x={"x":X_a},
                    #y=y_a,
                    y=y_a,
                    batch_size=1000,
                    num_epochs=None,
                    shuffle=True)
my_nn_esti.train(
                input_fn=train_input_fn,
                steps=100000)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./sav/model.ckpt.
INFO:tensorflow:loss = 0.5110930090125245, step = 1
INFO:tensorflow:global_step/sec: 184.808
INFO:tensorflow:loss = 0.17830758441091996, step = 101 (0.548 sec)
INFO:tensorflow:global_step/sec: 194.183
INFO:tensorflow:loss = 0.17963103783548287, step = 201 (0.516 sec)
INFO:tensorflow:global_step/sec: 177.588
INFO:tensorflow:loss = 0.15480556241889104, step = 301 (0.559 sec)
INFO:tensorflow:global_step/sec: 198.745
INFO:tensorflow:loss = 0.11547830668127947, step = 401 (0.503 sec)
INFO:tensorflow:global_step/sec: 115.18
INFO:tensorflow:loss = 0.2093523238377539, step = 501 (0.872 sec)
INFO:tensorflow:global_step/sec: 188.02
INFO:tensorflow:loss = 0.15660968711699433, step = 601 (0.533 sec)
INFO:tensorflow:global_step/sec: 95.0935
INFO:tensorflow:loss = 0.0708209619212845, step = 701 (1.049 sec)
INFO:tensorflow:global_step/sec: 142.681
INFO:tensorflow:loss = 0.05089265562283002, ste

INFO:tensorflow:global_step/sec: 182.29
INFO:tensorflow:loss = 0.059181954642147636, step = 7501 (0.548 sec)
INFO:tensorflow:global_step/sec: 110.208
INFO:tensorflow:loss = 0.06342580811297006, step = 7601 (0.906 sec)
INFO:tensorflow:global_step/sec: 72.9157
INFO:tensorflow:loss = 0.069383940738786, step = 7701 (1.368 sec)
INFO:tensorflow:global_step/sec: 241.563
INFO:tensorflow:loss = 0.04911954998653101, step = 7801 (0.418 sec)
INFO:tensorflow:global_step/sec: 234.78
INFO:tensorflow:loss = 0.03657881838255323, step = 7901 (0.425 sec)
INFO:tensorflow:global_step/sec: 235.639
INFO:tensorflow:loss = 0.038455880037760716, step = 8001 (0.426 sec)
INFO:tensorflow:global_step/sec: 77.4676
INFO:tensorflow:loss = 0.038588585040234914, step = 8101 (1.290 sec)
INFO:tensorflow:global_step/sec: 183.188
INFO:tensorflow:loss = 0.039857191569471914, step = 8201 (0.547 sec)
INFO:tensorflow:global_step/sec: 97.0267
INFO:tensorflow:loss = 0.04888372530392323, step = 8301 (1.029 sec)
INFO:tensorflow:glo

INFO:tensorflow:global_step/sec: 128.481
INFO:tensorflow:loss = 0.05150475620244922, step = 15001 (0.776 sec)
INFO:tensorflow:global_step/sec: 227.716
INFO:tensorflow:loss = 0.038363325410753615, step = 15101 (0.439 sec)
INFO:tensorflow:global_step/sec: 142.625
INFO:tensorflow:loss = 0.04223681053349551, step = 15201 (0.701 sec)
INFO:tensorflow:global_step/sec: 126.282
INFO:tensorflow:loss = 0.04115911288299533, step = 15301 (0.792 sec)
INFO:tensorflow:global_step/sec: 72.0534
INFO:tensorflow:loss = 0.040871923982111785, step = 15401 (1.388 sec)
INFO:tensorflow:global_step/sec: 88.5765
INFO:tensorflow:loss = 0.042749391654023305, step = 15501 (1.129 sec)
INFO:tensorflow:global_step/sec: 230.836
INFO:tensorflow:loss = 0.04876166927881129, step = 15601 (0.434 sec)
INFO:tensorflow:global_step/sec: 202.115
INFO:tensorflow:loss = 0.0663294447368661, step = 15701 (0.491 sec)
INFO:tensorflow:global_step/sec: 163.284
INFO:tensorflow:loss = 0.07103973045535407, step = 15801 (0.615 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 167.449
INFO:tensorflow:loss = 0.036107201750205785, step = 22501 (0.596 sec)
INFO:tensorflow:global_step/sec: 219.599
INFO:tensorflow:loss = 0.04034604314277435, step = 22601 (0.456 sec)
INFO:tensorflow:global_step/sec: 213.631
INFO:tensorflow:loss = 0.05730658367693342, step = 22701 (0.468 sec)
INFO:tensorflow:global_step/sec: 194.335
INFO:tensorflow:loss = 0.0566286546079216, step = 22801 (0.516 sec)
INFO:tensorflow:global_step/sec: 215.02
INFO:tensorflow:loss = 0.06158002327509547, step = 22901 (0.464 sec)
INFO:tensorflow:global_step/sec: 228.129
INFO:tensorflow:loss = 0.10218440573741717, step = 23001 (0.439 sec)
INFO:tensorflow:global_step/sec: 208.93
INFO:tensorflow:loss = 0.04747398290220696, step = 23101 (0.479 sec)
INFO:tensorflow:global_step/sec: 218.74
INFO:tensorflow:loss = 0.03554535997438354, step = 23201 (0.457 sec)
INFO:tensorflow:global_step/sec: 226.47
INFO:tensorflow:loss = 0.03531385041610099, step = 23301 (0.440 sec)
INFO:tensorflo

INFO:tensorflow:global_step/sec: 200.039
INFO:tensorflow:loss = 0.05231095110768761, step = 30001 (0.500 sec)
INFO:tensorflow:global_step/sec: 217.816
INFO:tensorflow:loss = 0.057343039536321375, step = 30101 (0.460 sec)
INFO:tensorflow:global_step/sec: 216.558
INFO:tensorflow:loss = 0.10660005457959383, step = 30201 (0.462 sec)
INFO:tensorflow:global_step/sec: 216.141
INFO:tensorflow:loss = 0.04713743041485175, step = 30301 (0.462 sec)
INFO:tensorflow:global_step/sec: 191.23
INFO:tensorflow:loss = 0.03256467693949834, step = 30401 (0.524 sec)
INFO:tensorflow:global_step/sec: 217.96
INFO:tensorflow:loss = 0.03647683766164708, step = 30501 (0.459 sec)
INFO:tensorflow:global_step/sec: 215.672
INFO:tensorflow:loss = 0.03553555081375809, step = 30601 (0.464 sec)
INFO:tensorflow:global_step/sec: 221.551
INFO:tensorflow:loss = 0.03822310562209365, step = 30701 (0.452 sec)
INFO:tensorflow:global_step/sec: 211.408
INFO:tensorflow:loss = 0.054633902269768754, step = 30801 (0.473 sec)
INFO:tenso

...

INFO:tensorflow:global_step/sec: 211.889
INFO:tensorflow:loss = 0.03692825248990666, step = 67401 (0.471 sec)
INFO:tensorflow:global_step/sec: 227.029
INFO:tensorflow:loss = 0.04567466994930755, step = 67501 (0.440 sec)
INFO:tensorflow:global_step/sec: 226.401
INFO:tensorflow:loss = 0.05478064421203382, step = 67601 (0.440 sec)
INFO:tensorflow:global_step/sec: 225.896
INFO:tensorflow:loss = 0.05114663058968348, step = 67701 (0.444 sec)
INFO:tensorflow:global_step/sec: 226.694
INFO:tensorflow:loss = 0.0675942074045534, step = 67801 (0.440 sec)
INFO:tensorflow:global_step/sec: 223.901
INFO:tensorflow:loss = 0.10857964725020318, step = 67901 (0.448 sec)
INFO:tensorflow:global_step/sec: 225.661
INFO:tensorflow:loss = 0.03966113660578579, step = 68001 (0.444 sec)
INFO:tensorflow:global_step/sec: 225.14
INFO:tensorflow:loss = 0.03575252175070899, step = 68101 (0.444 sec)
INFO:tensorflow:global_step/sec: 225.135
INFO:tensorflow:loss = 0.03291314576120564, step = 68201 (0.444 sec)
INFO:tensorf

INFO:tensorflow:global_step/sec: 211.235
INFO:tensorflow:loss = 0.05743105949514702, step = 74901 (0.474 sec)
INFO:tensorflow:global_step/sec: 225.971
INFO:tensorflow:loss = 0.06574787340525991, step = 75001 (0.442 sec)
INFO:tensorflow:global_step/sec: 227.516
INFO:tensorflow:loss = 0.12809633663807365, step = 75101 (0.440 sec)
INFO:tensorflow:global_step/sec: 223.146
INFO:tensorflow:loss = 0.04398198137099311, step = 75201 (0.448 sec)
INFO:tensorflow:global_step/sec: 225.62
INFO:tensorflow:loss = 0.034100629348937754, step = 75301 (0.444 sec)
INFO:tensorflow:global_step/sec: 226.847
INFO:tensorflow:loss = 0.0348289068828452, step = 75401 (0.440 sec)
INFO:tensorflow:global_step/sec: 226.355
INFO:tensorflow:loss = 0.03843537669258238, step = 75501 (0.444 sec)
INFO:tensorflow:global_step/sec: 226.701
INFO:tensorflow:loss = 0.04506063011205636, step = 75601 (0.440 sec)
INFO:tensorflow:global_step/sec: 226.731
INFO:tensorflow:loss = 0.05362939392257775, step = 75701 (0.440 sec)
INFO:tensor

INFO:tensorflow:global_step/sec: 211.26
INFO:tensorflow:loss = 0.03932540080951609, step = 82401 (0.472 sec)
INFO:tensorflow:global_step/sec: 227.517
INFO:tensorflow:loss = 0.035495500876078175, step = 82501 (0.439 sec)
INFO:tensorflow:global_step/sec: 219.394
INFO:tensorflow:loss = 0.03871685235672463, step = 82601 (0.456 sec)
INFO:tensorflow:global_step/sec: 224.417
INFO:tensorflow:loss = 0.037035598937337905, step = 82701 (0.445 sec)
INFO:tensorflow:global_step/sec: 224.677
INFO:tensorflow:loss = 0.04273100507993088, step = 82801 (0.446 sec)
INFO:tensorflow:global_step/sec: 226.963
INFO:tensorflow:loss = 0.05628751645595235, step = 82901 (0.441 sec)
INFO:tensorflow:global_step/sec: 224.128
INFO:tensorflow:loss = 0.0544363481670981, step = 83001 (0.445 sec)
INFO:tensorflow:global_step/sec: 225.008
INFO:tensorflow:loss = 0.05797795040793316, step = 83101 (0.444 sec)
INFO:tensorflow:global_step/sec: 225.685
INFO:tensorflow:loss = 0.07707787073156248, step = 83201 (0.444 sec)
INFO:tenso

INFO:tensorflow:global_step/sec: 212.543
INFO:tensorflow:loss = 0.04042044298563158, step = 89901 (0.468 sec)
INFO:tensorflow:global_step/sec: 227.44
INFO:tensorflow:loss = 0.050284238596720515, step = 90001 (0.440 sec)
INFO:tensorflow:global_step/sec: 228.049
INFO:tensorflow:loss = 0.058445023187563956, step = 90101 (0.440 sec)
INFO:tensorflow:global_step/sec: 227.809
INFO:tensorflow:loss = 0.05046241048226583, step = 90201 (0.439 sec)
INFO:tensorflow:global_step/sec: 226.454
INFO:tensorflow:loss = 0.0736500955894664, step = 90301 (0.442 sec)
INFO:tensorflow:global_step/sec: 226.279
INFO:tensorflow:loss = 0.05747352907808359, step = 90401 (0.442 sec)
INFO:tensorflow:global_step/sec: 225.883
INFO:tensorflow:loss = 0.03360873766971769, step = 90501 (0.442 sec)
INFO:tensorflow:global_step/sec: 225.351
INFO:tensorflow:loss = 0.03202583901031531, step = 90601 (0.444 sec)
INFO:tensorflow:global_step/sec: 224.839
INFO:tensorflow:loss = 0.037290420823276015, step = 90701 (0.444 sec)
INFO:tens

INFO:tensorflow:global_step/sec: 126.835
INFO:tensorflow:loss = 0.05509944230907881, step = 97401 (0.794 sec)
INFO:tensorflow:global_step/sec: 132.699
INFO:tensorflow:loss = 0.0802287813064101, step = 97501 (0.752 sec)
INFO:tensorflow:global_step/sec: 132.701
INFO:tensorflow:loss = 0.04621394660065618, step = 97601 (0.749 sec)
INFO:tensorflow:global_step/sec: 133.785
INFO:tensorflow:loss = 0.03534899269315372, step = 97701 (0.752 sec)
INFO:tensorflow:global_step/sec: 131.483
INFO:tensorflow:loss = 0.031950189366503436, step = 97801 (0.757 sec)
INFO:tensorflow:global_step/sec: 132.547
INFO:tensorflow:loss = 0.03813495981034519, step = 97901 (0.758 sec)
INFO:tensorflow:global_step/sec: 131.085
INFO:tensorflow:loss = 0.041976395222728925, step = 98001 (0.758 sec)
INFO:tensorflow:global_step/sec: 132.586
INFO:tensorflow:loss = 0.0528541361101891, step = 98101 (0.756 sec)
INFO:tensorflow:global_step/sec: 131.028
INFO:tensorflow:loss = 0.05458127496076759, step = 98201 (0.768 sec)
INFO:tenso

In [10]:
print (my_rmse_list)

[]
